In [1]:
import os
import sys
import json
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import GoogleMapsGeocoding as gmaps
import importlib
import LLAMAconnect
import subprocess
import shutil
from pprint import pprint
import random
import vehParameters

In [70]:
net_path = 'osm.net.xml'
add_path = 'osm_stops.add.xml'
net = sumolib.net.readNet(net_path)
parkingAreas = list(sumolib.output.parse(add_path, "parkingArea"))

In [4]:
radius = 150

In [5]:
def has_parking_spot(lanes, parkingAreas):
    # Example of parkingArea:
    # <parkingArea id="pa-1046248579#0" lane="-1046248579#0_0" roadsideCapacity="94" length="5.00"/>
    # Returns parkingArea id if there is a parking spot in the lane
    lane_ids = [lane.getID() for lane in lanes]
    for park in parkingAreas:
        if park.lane in lane_ids:
            return park.id
    return False

In [6]:
def getClosestEdges(lat, lon, radius, maxEdges=10):
    # Gets the 10 closest edges to the given lat, lon
    x, y = net.convertLonLat2XY(lon, lat)
    edges = net.getNeighboringEdges(x, y, radius)
    closestEdges = []
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])

        ## Checking if the edge found can be used by passenger car
        for dist, edge in distanceAndEdges:
            if edge.allows('passenger'):
                closestEdges.append(edge)

    if len(edges) == 0:
        print(f'No edges found for {lat}, {lon}')
    return closestEdges

In [7]:
def getParkingSpot(lat, lon, radius, parkingAreas):
    # Get the parking spot closest to the given lat, lon
    # Used to set stops for the vehicles

    edges = getClosestEdges(lat, lon, radius)
    # Look for parking spots
    for i in range(len(edges)):
        parking_spot = has_parking_spot(edges[i].getLanes(), parkingAreas)
        if parking_spot:
            return parking_spot
    print("No parking spot found:", lat, lon)
    return None

In [8]:
def getPath(location_time_list, parkingAreas):
    # All that is needed to create the trip are the stops (parking areas) and the start and end edges.
    # The duarouter is responsible for finding the path between the edges going through the stops.
    # Here, we get the edges and stops that are going to be sent to LLAMA to create the trip.

    # 'coordinates' is a list of tuples with the latitude and longitude of the points of interest, for example IC, FEEC, IC means that
    # the vehicle will start from IC, stop at a parking lot close to FEEC, and then back to IC.
    # The first and last coordinates should be edges and the others should be parking spots.

    # Departure for 7 is 0, 8 is 100, 9 is 200 and so on
    stop_durations = []
    departures = list(location_time_list.keys())
    stop_durations.append(-1) # Indicates this is an edge and not a parking spot

    path = []
    locations = list(location_time_list.values())
    home = getClosestEdges(*locations[0], radius)[0].getID()
    path.append(home)
    
    for i in range(1, len(locations)-1):
        stop_durations.append(100 * (departures[i] - departures[i - 1]))
        path.append(getParkingSpot(*locations[i], radius, parkingAreas))
    
    path.append(home)
    stop_durations.append(-1)
    
    return path, stop_durations

In [9]:
def pathToXML(path, vehicleID, veh_type, departure_time, stop_durations):
    # Converts the path to the XML format that LLAMA understands
    xml = f'<trip id="{vehicleID}" type="{veh_type}" depart="{departure_time}" from="{path[0]}" to="{path[-1]}">\n'
    for i in range(1, len(path)-1):
        xml += f'\t<stop parkingArea="{path[i]}" duration="{stop_durations[i]}"/>\n'
    xml += '</trip>'
    return xml

Getting the routine for the students described

In [ ]:

# It is important not to use short names of institutes, like IB for institute of biology, because the LLM will not be able to assign the students to the correct institute.
places ={
    'leisure': ["bar", "movie_theater", "shopping_mall"],
    'eating': ["restaurant", "bakery", "cafe"],
    'shopping': ["store", "drugstore", "supermarket"],
    'sports': ["gym"],
    'institute': ["Institute of Biology", "Institute of Computing", "Institute of Geociences", "Institute of Philosophy and Human Sciences", "Institute of Chemistry"],
    'university': ["University of Campinas"],
}

student_info = "The student is geography student. Today, the student is going to have lunch at a restaurant and the student is going to the supermarket."
importlib.reload(LLAMAconnect)
responses = LLAMAconnect.generate_response_trips(student_info, places, number_of_trips=5) # number of trips defaults to 5
pprint(json.loads(responses[0]))

  0%|          | 0/5 [00:00<?, ?it/s]

 40%|████      | 2/5 [00:01<00:01,  1.68it/s]

Error: Invalid location generated 'Institute of Geography and Human Sciences'
Invalid response. Generating a new one.
Error: Invalid location generated 'Institute of Geography and Human Sciences'
Invalid response. Generating a new one.


 60%|██████    | 3/5 [00:02<00:02,  1.13s/it]

Error: Invalid location generated 'Institute of Geography and Human Sciences'
Invalid response. Generating a new one.


 80%|████████  | 4/5 [00:04<00:01,  1.12s/it]

Error: The response is missing some hours
Invalid response. Generating a new one.


100%|██████████| 5/5 [00:05<00:00,  1.03s/it]

{'10': {'activity': 'study', 'location': 'Institute of Geociences'},
 '11': {'activity': 'study', 'location': 'Institute of Geociences'},
 '12': {'activity': 'lunch', 'location': 'restaurant'},
 '13': {'activity': 'study', 'location': 'Institute of Geociences'},
 '14': {'activity': 'study', 'location': 'Institute of Geociences'},
 '15': {'activity': 'study', 'location': 'Institute of Geociences'},
 '16': {'activity': 'practicing sports', 'location': 'gym'},
 '17': {'activity': 'shopping', 'location': 'store'},
 '18': {'activity': 'shopping', 'location': 'supermarket'},
 '19': {'activity': 'dinner', 'location': 'home'},
 '20': {'activity': 'leisure', 'location': 'movie_theater'},
 '21': {'activity': 'relax', 'location': 'home'},
 '22': {'activity': 'relax', 'location': 'home'},
 '23': {'activity': 'bed', 'location': 'home'},
 '7': {'activity': 'wake up', 'location': 'home'},
 '8': {'activity': 'study', 'location': 'Institute of Geociences'},
 '9': {'activity': 'study', 'location': 'Inst

In [38]:
pprint(json.loads(responses[4]))

{'10': {'activity': 'study', 'location': 'Institute of Geociences'},
 '11': {'activity': 'study', 'location': 'Institute of Geociences'},
 '12': {'activity': 'lunch', 'location': 'restaurant'},
 '13': {'activity': 'shopping', 'location': 'store'},
 '14': {'activity': 'study', 'location': 'Institute of Geociences'},
 '15': {'activity': 'study', 'location': 'Institute of Geociences'},
 '16': {'activity': 'study', 'location': 'Institute of Geociences'},
 '17': {'activity': 'sports', 'location': 'gym'},
 '18': {'activity': 'dinner', 'location': 'restaurant'},
 '19': {'activity': 'leisure', 'location': 'home'},
 '20': {'activity': 'leisure', 'location': 'home'},
 '21': {'activity': 'leisure', 'location': 'home'},
 '22': {'activity': 'leisure', 'location': 'home'},
 '23': {'activity': 'bed', 'location': 'home'},
 '7': {'activity': 'wake up', 'location': 'home'},
 '8': {'activity': 'study', 'location': 'Institute of Geociences'},
 '9': {'activity': 'study', 'location': 'Institute of Geocience

Generating different houses for students based on a random number generator in the where students might live

In [39]:
def getHome():
    # top right -22.815539, -47.076272
    # top left: -22.818517, -47.079520
    # bottom left: -22.826410, -47.072591
    # bottom right: -22.823439, -47.068828
    X_MIN = 22815539
    X_MAX = 22826410
    Y_MIN = 47068828
    Y_MAX = 47079520
    while True:
        random_x = random.randint(X_MIN, X_MAX)
        random_y = random.randint(Y_MIN, Y_MAX)
        lat = - random_x / 10**6
        lon = - random_y / 10**6
        if (lat, lon) in coords.values():
            continue
        break
    return lat, lon

In [ ]:
def getCoords(trip, sulfixo, institutes, start_radius, step_radius, limit_radius, siglas=None):
    # Returns a dictionary with the latitude and longitude of the locations of interest
    # The suffix is the name of the state, city and neighborhood that will be added to the end of each location to improve the search
    # start_radius is the initial radius of the search, step_radius is the amount that will be added to the radius if the location is not found and limit_radius is the maximum radius that will be used. After that, the student will choose not to leave the place he is at.
    # 'siglas' is a dictionary that maps the name of the location to a shorter name, if needed
    coords = {}
    importlib.reload(gmaps)
    for i in range(len(trip)):
        local = trip[f'{i + 7}']['location']
        local_comp = local + ", " + sulfixo

        if local in coords.keys():
            continue
        
        if local == 'home':
            coords['home'] = (-22.821026, -47.072902)
            continue
        elif local in institutes:
                result = gmaps.geocode_address(local_comp)
                lat, lon = result[0]['latitude'], result[0]['longitude']
        else:
            previous = coords[trip[f'{i + 6}']['location']]
            result = gmaps.find_nearby_building(previous[0], previous[1], local, radius=start_radius)
            expanded = radius
            while result is False:
                print(f"Could not find {local} in a radius of {expanded} meters. Trying again with a larger radius.")
                expanded += step_radius
                if expanded > limit_radius:
                    print(f"Could not find {local} in a radius of {limit_radius} meters. The student will not leave the place he is currently at.")
                    result = [{'latitude': previous[0], 'longitude': previous[1]}]
                    break

                result = gmaps.find_nearby_building(previous[0], previous[1], local, radius=expanded)

            random.shuffle(result) # Randomize the results to avoid always getting the absolute closest building
            lat, lon = result[0]['latitude'], result[0]['longitude']
        
        if siglas is not None:
            try:
                coords[f'{siglas[local]}'] = (lat, lon)
            except KeyError:
                coords[f'{local}'] = (lat, lon)
        else:
            coords[f'{local}'] = (lat, lon)
        
    return coords

In [41]:
def coordsToTrip(trip, coords):
    location_time = {}
    home = coords['home'] # Gets a random home location for each student
    last = home
    location_time[7] = last
    
    for j in range(1, len(trip)):
        local_coords = coords[trip[f'{j + 7}']['location']]
        if local_coords != last:
            location_time[j + 7] = local_coords
            last = local_coords

    return location_time

Getting the latitude and longitude for each of the locations

In [ ]:
sufixo = "Barão Geraldo, Campinas, SP, Brazil"
trips = {}
location_time_list = []
for i in range(len(responses)):
    json_response = json.loads(responses[i])
    trips[i] = json_response
    print(f"Getting coords for trip {i + 1}:")
    coords = getCoords(trips[i], sufixo, places['institute'], start_radius=100, step_radius=100, limit_radius=1500)
    location_time_list.append(coordsToTrip(trips[i], coords))
    print("")


Getting coords for trip 1:
Could not find restaurant in a radius of 150 meters. Trying again with a larger radius.
Could not find restaurant in a radius of 250 meters. Trying again with a larger radius.
Could not find gym in a radius of 150 meters. Trying again with a larger radius.
Could not find gym in a radius of 250 meters. Trying again with a larger radius.
Could not find store in a radius of 150 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 150 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 250 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 350 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 450 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 550 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 650 meters. Trying again with a larger radius.
Could not find mov

In [57]:
pprint(location_time_list)

[{7: (-22.821026, -47.072902),
  8: (-22.8133258, -47.0691729),
  12: (-22.8122661, -47.0663524),
  13: (-22.8133258, -47.0691729),
  16: (-22.810389, -47.0696112),
  17: (-22.8084323, -47.0701713),
  18: (-22.810265, -47.0767833),
  19: (-22.821026, -47.072902)},
 {7: (-22.821026, -47.072902),
  8: (-22.8133258, -47.0691729),
  9: (-22.821026, -47.072902),
  10: (-22.8133258, -47.0691729),
  12: (-22.8122661, -47.0663524),
  13: (-22.810265, -47.0767833),
  14: (-22.8133258, -47.0691729),
  17: (-22.810389, -47.0696112),
  18: (-22.8102069, -47.07551640000001),
  19: (-22.821026, -47.072902)},
 {7: (-22.821026, -47.072902),
  8: (-22.8133258, -47.0691729),
  12: (-22.8122661, -47.0663524),
  13: (-22.810265, -47.0767833),
  14: (-22.8150093, -47.0683385),
  18: (-22.8122661, -47.0663524),
  20: (-22.8090377, -47.0687294),
  21: (-22.821026, -47.072902)},
 {7: (-22.821026, -47.072902),
  8: (-22.8133258, -47.0691729),
  12: (-22.8122661, -47.0663524),
  13: (-22.810265, -47.0767833),
 

Taking the locations and turning them into a XML trips, which are written to PATHGEN.trips.xml

In [80]:
randomtrips_path = "/usr/share/sumo/tools/randomTrips.py"
def randomtrips_getArgs(net_path, additional_path, v_class, n_trips):
    args = [
        "-n", net_path,
        "-o", "randtrips.trips.xml",
        "--additional", additional_path,
        "--vclass", v_class,
        "--vehicle-class", v_class,
        "-e", str(n_trips)
    ]
    return args

In [81]:
def getRandomTrips(net_path, add_path, vclass, n_trips):
    # Run the randomTrips.py to create the trips
    subprocess.run(['python3', randomtrips_path] + randomtrips_getArgs(net_path, add_path, vclass, n_trips), check=True)

In [112]:
def parseTripXML(location_time_list, parkingAreas, departure_times, styles, veh_types_per_student, n_vtypes=5, rand_trips=None):
    # Creates the XML for the trips
    # 'departure_times' is a list with the departure times for each trip
    # 'styles' is a list with the styles of the vehicles. The current supported styles are "agg" for aggressive and "norm" for normal
    # 'veh_types_per_student' is a list with the vehicle types for each student
    # 'n_vtypes' is the number of vTypes created for each style
    # 'rand_trips' is a file containing the random trips
    
    importlib.reload(vehParameters)

    xml = '<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n'

    xml += '\n<!-- BEGIN - LLM Generated trips -->\n'

    xml += '\n<!-- Vehicles -->\n'
    vehdists = vehParameters.generateVehicleTypes(styles, n_vtypes)
    xml += vehParameters.parseVehiclesXML(vehdists, styles)

    xml += '\n'
    xml += '<!-- Trips -->\n'
    for i in range(len(location_time_list)):
        path, stop_durations = getPath(location_time_list[i], parkingAreas)
        xml += pathToXML(path, f'veh{i + 1}', veh_types_per_student[i], departure_times[i], stop_durations) + '\n'

    xml += '\n\n <!-- END - LLM Generated trips -->\n\n'

    xml += '<!-- BEGIN - Random Trips -->\n\n'

    if rand_trips:
        start_read = False
        with open(rand_trips, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('<vType'):
                    start_read = True
                if start_read:
                    if line.startswith('</routes>'):
                        break
                    xml += line + '\n'

    xml += '\n</routes>'
    
    with open('PATHGEN.trips.xml', 'w') as f:
        f.write(xml)
    
    return xml

In [113]:
departure_times = []
for i in range(len(location_time_list)):
    departure_times.append((list(location_time_list[i].keys())[0] - 7) * 20)

styles = ["agg", "norm"] # Currently only two styles, but can be expanded
n_vtypes = 5 
veh_style_per_student = []
for i in range(len(location_time_list)):
    if i < len(location_time_list) / 2:
        veh_style_per_student.append('agg')
    else: 
        veh_style_per_student.append('norm')   

xml = parseTripXML(location_time_list, parkingAreas, departure_times, styles, veh_style_per_student, n_vtypes, rand_trips='randtrips.trips.xml')
print(xml)

<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">

<!-- BEGIN - LLM Generated trips -->

<!-- Vehicles -->
<vTypeDistribution id="agg">
	<vType id="v_agg0" minGap="0.23" accel="3.03" decel="4.05" startupDelay="0.29" sigma="0.77" tau="1.51" maxSpeed="119.89" speedFactor="1.82" lcStrategic="6.76" lcCooperative="0.07" lcSpeedGain="7.18" lcKeepRight="1.67" lcOvertakeRight="0.62" lcSpeedGainLookahead="3.63" lcOvertakeDeltaSpeedFactor="0.64" lcPushy="0.79" lcAssertive="3.11" lcImpatience="0.53" lcTimeToImpatience="8.75" lcLaneDiscipline="2.17" lcSigma="0.95" lcAccelLat="1.41" probability="0.03832543322829782">
		<param key="device.rerouting.probability" value="1.0"/>
		<param key="device.rerouting.adaptation-steps" value="18"/>
		<param key="device.rerouting.adaptation-interval" value="10"/>
	</vType>
	<vType id="v_agg1" minGap="0.44" accel="3.78" decel="3.44" startupDelay="0.5" sigma="0.82" tau="1.31" maxSpe

Getting alternative routes for every route found

In [ ]:
## Arguments to get alternative routes
duaiterate_path = "/usr/share/sumo/tools/assign/duaIterate.py"
def duaiterate_getArgs(net_path, trips_path, additional_path, iterations):
    args = [
        "-n", net_path,
        "-t", trips_path,
        "--additional", additional_path,
        "duarouter--additional-files", additional_path,
        "-l", str(iterations)
    ]
    return args

In [ ]:
def getAltRoutes(net_path, trips_path, additional_path, iterations):
    # Run the duarouter to find alternative routes
    # Currently this only works for less than 10 iterations because of file naming
    for file in os.listdir('.'):
        if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
            os.remove(file)
    
    try:
        subprocess.run(['python3', duaiterate_path] + duaiterate_getArgs(net_path, trips_path, additional_path, iterations), check=True)
        shutil.move(f'00{iterations-1}/PATHGEN_00{iterations-1}.rou.alt.xml', f'PATHGEN_00{iterations-1}.rou.alt.xml')
        for i in range(iterations):
            if os.path.exists(f'00{i}'):
                shutil.rmtree(f'00{i}')
        for file in os.listdir('.'):
            if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
                os.rename(file, f'PATHGEN.rou.alt.xml')
    except subprocess.CalledProcessError as e:
        print(f"{e}")

In [ ]:
getAltRoutes(net_path, 'PATHGEN.trips.xml', add_path, 5)

> Executing step 0
>> Running router on PATHGEN.trips.xml
>>> Begin time: 2024-11-01 20:29:07.339982
>>> End time: 2024-11-01 20:29:08.163089
>>> Duration: 0:00:00.823107
<<
>> Running simulation
>>> Begin time: 2024-11-01 20:29:08.163183
>>> End time: 2024-11-01 20:29:10.027174
>>> Duration: 0:00:01.863991
<<
< Step 0 ended (duration: 0:00:02.687891)
------------------

> Executing step 1
>> Running router on 000/PATHGEN_000.rou.alt.xml
>>> Begin time: 2024-11-01 20:29:10.033247
>>> End time: 2024-11-01 20:29:10.842358
>>> Duration: 0:00:00.809111
<<
>> Running simulation
>>> Begin time: 2024-11-01 20:29:10.842498
>>> End time: 2024-11-01 20:29:12.644832
>>> Duration: 0:00:01.802334
<<
< Step 1 ended (duration: 0:00:02.611917)
------------------

> Executing step 2
>> Running router on 001/PATHGEN_001.rou.alt.xml
>>> Begin time: 2024-11-01 20:29:12.658072
>>> End time: 2024-11-01 20:29:13.441470
>>> Duration: 0:00:00.783398
<<
>> Running simulation
>>> Begin time: 2024-11-01 20:29:13.

To use this file, just set PATHGEN.rou.alt.xml as a trips file in osm.sumocfg